In [15]:
import cv2
import numpy as np
import os
import sys
import tensorflow as tf
from datetime import datetime
from sklearn.model_selection import train_test_split
from tensorflow.python.ops.gen_math_ops import mod
from PIL import Image, ImageFilter

In [16]:
EPOCHS = 3
IMG_WIDTH = 256
IMG_HEIGHT = 256
NUM_CATEGORIES = 6
TEST_SIZE = 0.5
GESTURE = {0:"ok", 1:"down", 2:"up", 3:"palm", 4:"fist", 5:"l"}

In [17]:
def load_data(data_dir):
    """
    Load image data from directory `data_dir`.
    Assume `data_dir` has one directory named after each category, numbered
    0 through NUM_CATEGORIES - 1. Inside each category directory will be some
    number of image files.
    Return tuple `(images, labels)`. `images` should be a list of all
    of the images in the data directory, where each image is formatted as a
    numpy ndarray with dimensions IMG_WIDTH x IMG_HEIGHT x 3. `labels` should
    be a list of integer labels, representing the categories for each of the
    corresponding `images`.
    """
    images = []
    labels = []
    
    for dir in range(0, NUM_CATEGORIES):
        # get path for each gesture like "/home/arpine/Desktop/data/0":  
        d = os.path.join(data_dir, f"{str(dir)}")
        # os.listdir(d) return the list of all names of images in that folder
        for image_path in os.listdir(d):
            # get the full path of specific image 
            full_path = os.path.join(data_dir, f"{str(dir)}", image_path)
            # Returns an image that is loaded from the specified file
            image = cv2.imread(full_path, )
            # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            # cv2.imshow("im", image)
            # get dimension for each image
            dim = (IMG_WIDTH, IMG_HEIGHT)
            # resized the image
            image_resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
            
            # add image and their directory name to images and labels list
            images.append(image_resized)
            labels.append(dir)
    
    return images, labels

In [18]:
start_time = datetime.now()  
print("Loading ===========")  
images, labels = load_data("/home/arpine/Desktop/Gesture/poqr")  
finish_loading_time = datetime.now()
print("Images load time: ", finish_loading_time - start_time)

Loading ===========
Images load time:  0:00:00.951040


In [19]:
def get_model():
    """
    Returns a compiled convolutional neural network model. Assume that the
    `input_shape` of the first layer is `(IMG_WIDTH, IMG_HEIGHT, 3)`.
    The output layer should have `NUM_CATEGORIES` units, one for each category.
    """
    # Create a convolutional neural network
    model = tf.keras.models.Sequential(
        [
        # Convolutional layer. Learn 32 filters using a 3x3 kernel
        tf.keras.layers.Conv2D(
            32, (5, 5), activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)
        ),
        # Max-pooling layer, using 2x2 pool size
        tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(
            64, (3, 3), activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)
        ),
        # Max-pooling layer, using 2x2 pool size
        tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(
            128, (3, 3), activation='relu', input_shape=((IMG_WIDTH), (IMG_HEIGHT), 3)
        ),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(
            256, (3, 3), activation='relu', input_shape=((IMG_WIDTH), (IMG_HEIGHT), 3)
        ),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
        
        tf.keras.layers.Flatten(),
        # Add a hidden layer with dropout
        tf.keras.layers.Dense(128, activation='relu'),
        #tf.keras.layers.Dropout(0.3),
        # Add an output layer with output units for all 6 gestures
        tf.keras.layers.Dense(NUM_CATEGORIES, activation='softmax')
    ])

    # Train neural network
    model.compile(
        optimizer='adam',
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

In [20]:
labels = tf.keras.utils.to_categorical(labels)
x_train, x_test, y_train, y_test = train_test_split(
    np.array(images), np.array(labels), test_size=TEST_SIZE)

# Get a compiled neural network
model = get_model()

# Fit model on training data
model.fit(x_train, y_train, batch_size=64, epochs=EPOCHS)

# Evaluate neural network performance
model.evaluate(x_test, y_test, verbose=2)
fitting_time = datetime.now()

print("NN fit time: ", fitting_time - finish_loading_time)

Epoch 1/3
21/21 [==============================] - 55s 3s/step - loss: 34.3979 - accuracy: 0.6501
Epoch 2/3
21/21 [==============================] - 56s 3s/step - loss: 0.0092 - accuracy: 0.9992
Epoch 3/3
21/21 [==============================] - 56s 3s/step - loss: 0.0019 - accuracy: 1.0000
41/41 - 11s - loss: 0.0093 - accuracy: 0.9977
NN fit time:  0:03:04.191403


In [26]:
video = cv2.VideoCapture(0)
    
while True:
        # Capture the video frame
        ret, img = video.read()

        # Display the resulting frame
        # to flip the video with 180 degree 
        image = cv2.flip(img, 1)
        
        # address = "/home/arpine/Desktop/Gesture/Frame0.png"
        # image = cv2.imwrite(address, image)
        image = cv2.imwrite('Frame'+str(0)+'.png', image)
        image_path = "Frame0.png"
        image = cv2.imread(image_path, 0)
        # save image for prediction
        # image = cv2.imread(image, 0)
        rest, thresh = cv2.threshold(image, 70, 255, cv2.THRESH_BINARY)
        _, contours = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        image = cv2.imwrite(image_path, thresh)
        # image = cv2.imwrite('Frame'+str(0)+'.png', image)
        # address = "/home/arpine/Desktop/Gesture/Frame0.png"
        # image = Image.open(address).convert("RGB")
        # image = image.filter(ImageFilter.Kernel(
        #         size=(3, 3),
        #         kernel=[-1, -1, -1, -1, 8, -1, -1, -1, -1],
        #         scale=1
        #     ))
        # image = image.save(address)
        # image = "Frame0.png"
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        dim = (IMG_WIDTH, IMG_HEIGHT)
        
        image = tf.keras.preprocessing.image.load_img(image, target_size=dim)
        # Converts a PIL Image instance to a Numpy array. Return a 3D Numpy array.
        input_arr = tf.keras.preprocessing.image.img_to_array(image)
        # Convert single image to a batch.
        input_arr = np.array([input_arr])
        input_arr = input_arr.astype('float32')/255
        # Generates output predictions for the input samples. Return Numpy array(s) of predictions.
        predictions = model.predict(input_arr)
        # Return the index_array of the maximum values along an axis.
        pre_class = np.argmax(predictions, axis=-1)
        #print(GESTURE[pre_class[0]])
        text = GESTURE[pre_class[0]]
        font = cv2.FONT_HERSHEY_SIMPLEX
        image = cv2.flip(img, 1)
        cv2.putText(image, 
                text, 
                (50, 50), 
                font, 2, 
                (0, 0, 0), 
                2, 
                cv2.LINE_4)
        cv2.imshow('video', image)
    

        # the 'q' button is set as the
        # quitting button you may use any
        # desired button of your choice
        k = cv2.waitKey(0)
        # if cv2.waitKey(10) and 0xFF == ord('q'):
        if k == ord('q'):
                break
        #     break
        
cv2.destroyAllWindows()
video.release()


error: OpenCV(4.5.2) /tmp/pip-req-build-eirhwqtr/opencv/modules/imgcodecs/src/loadsave.cpp:721: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'


In [ ]:
# import cv2
# import numpy as np
# import os
# import sys
# import tensorflow as tf
# from datetime import datetime
# from sklearn.model_selection import train_test_split
# from tensorflow.python.ops.gen_math_ops import mod

# from PIL import Image, ImageFilter

# EPOCHS = 10
# IMG_WIDTH = 640
# IMG_HEIGHT = 480
# NUM_CATEGORIES = 6
# TEST_SIZE = 0.5
# GESTURE = {0:"ok", 1:"down", 2:"up", 3:"palm", 4:"fist", 5:"l"}

# # Open image
# def load_data(data_dir):
#     """
#     Load image data from directory `data_dir`.
#     Assume `data_dir` has one directory named after each category, numbered
#     0 through NUM_CATEGORIES - 1. Inside each category directory will be some
#     number of image files.
#     Return tuple `(images, labels)`. `images` should be a list of all
#     of the images in the data directory, where each image is formatted as a
#     numpy ndarray with dimensions IMG_WIDTH x IMG_HEIGHT x 3. `labels` should
#     be a list of integer labels, representing the categories for each of the
#     corresponding `images`.
#     """
#     images = []
#     labels = []
    
#     for dir in range(0, NUM_CATEGORIES):
#         # get path for each gesture like "/home/arpine/Desktop/data/0":  
#         d = os.path.join(data_dir, f"{str(dir)}")
#         # os.listdir(d) return the list of all names of images in that folder
#         for image_path in os.listdir(d):
#             # get the full path of specific image 
#             full_path = os.path.join(data_dir, f"{str(dir)}", image_path)
#             image = Image.open(full_path).convert("RGB")
#             image = image.filter(ImageFilter.Kernel(
#                 size=(3, 3),
#                 kernel=[-1, -1, -1, -1, 8, -1, -1, -1, -1],
#                 scale=1
#             ))
#             image.save(full_path)
#             # Returns an image that is loaded from the specified file
#             image = cv2.imread(full_path, )
#             # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#             # cv2.imshow("im", image)
#             # get dimension for each image
#             dim = (IMG_WIDTH, IMG_HEIGHT)
#             # # resized the image
#             image_resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
            
#              # add image and their directory name to images and labels list
#             images.append(image_resized)
#             labels.append(dir)
    
#     return images, labels
#     # print(images)


# # Filter image according to edge detection kernel
# # image = image.filter(ImageFilter.Kernel(
# #     size=(3, 3),
# #     kernel=[-1, -1, -1, -1, 8, -1, -1, -1, -1],
# #     scale=1
# # ))

# # # Show resulting image
# # image.show()


# # cv2.waitKey(0)
# # cv2.destroyAllWindows()
# load_data("/home/arpine/Desktop/Gesture/test")

In [ ]:
import cv2
NUM_CATEGORIES = 6

GESTURE = {0:"ok", 1:"down", 2:"up", 3:"palm", 4:"fist", 5:"l"}

def load_data(data_dir):
    for dir in range(0, NUM_CATEGORIES):
    # adr = "/home/arpine/Desktop/Gesture/poqr/ok/bcbf8425-d850-11eb-9ec6-0ba2456509ee.png"
        d = os.path.join(data_dir, f"{str(dir)}")
        # os.listdir(d) return the list of all names of images in that folder
        for image_path in os.listdir(d):
        # get the full path of specific image 
            full_path = os.path.join(data_dir, f"{str(dir)}", image_path)
            image = cv2.imread(full_path, 0)
            rest, thresh = cv2.threshold(image, 150, 255, cv2.THRESH_BINARY)
            _, contours = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            cv2.imwrite(full_path, thresh)

In [ ]:
load_data("/home/arpine/Desktop/Gesture/poqr")